In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import json
import time
import matplotlib.pyplot as plt
import spotipy

from spotipy.oauth2 import SpotifyOAuth
from pandas.plotting import register_matplotlib_converters
from datetime import timedelta
register_matplotlib_converters()

plt.style.use('ggplot')

In [3]:
CLIENT_USERNAME = "willfurtado"
SPOTIPY_CLIENT_ID = "0c58b8f377294e1393b6ff20d1db34fc"
SPOTIPY_CLIENT_SECRET = "12fb3865a39343aba75ec4b118f6adf9"
SPOTIPY_REDIRECT_URI = "https://localhost:8888"

In [4]:
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, 
    client_id=SPOTIPY_CLIENT_ID, 
    client_secret=SPOTIPY_CLIENT_SECRET,
    username='willfurtado',
    redirect_uri=SPOTIPY_REDIRECT_URI))

In [20]:
def load_streaming_data(src):
    """
    Returns a Pandas DataFrame object with the current listening history 
    from Spotify data pull. The src argument is the folder in which the data is stored.
    """
    with open('personal_data/' + src + '/StreamingHistory0.json') as file:
        data0 = json.load(file)
    with open('personal_data/' + src + '/StreamingHistory1.json') as file:
        data1 = json.load(file)
    with open('personal_data/' + src + '/StreamingHistory2.json') as file:
        data2 = json.load(file)
        
    df0 = pd.DataFrame(data0)
    df1 = pd.DataFrame(data1)
    df2 = pd.DataFrame(data2)
    
    df = df0.append(df1, ignore_index=True).append(df2, ignore_index=True)
    df['secPlayed'] = round(df['msPlayed'] / 1000, 1)
    df = df.drop(columns=['msPlayed'])

    STRTIME_FORMAT = '%Y-%m-%d %H:%M'
    df['endTime'] = pd.to_datetime(df['endTime'], format=STRTIME_FORMAT)
    df['secPlayed_dt'] = df['secPlayed'].apply(lambda x: timedelta(seconds=x))
    df['startTime'] = df['endTime'] - df['secPlayed_dt']
    
    return df.dropna()

In [24]:
df = load_streaming_data("winter20")
df.head()

,endTime,artistName,trackName,secPlayed,secPlayed_dt,startTime
0,2019-12-04 00:20:00,Zane and Heath: Unfiltered,#13 - Our Hometown Hates Us Because of This,3542.1,00:59:02.100000,2019-12-03 23:20:57.900
1,2019-12-04 01:53:00,Kanye West,Every Hour,67.1,00:01:07.100000,2019-12-04 01:51:52.900
2,2019-12-04 02:36:00,Anne Hathaway,"I Dreamed A Dream - From ""Les Misérables""",277.5,00:04:37.500000,2019-12-04 02:31:22.500
3,2019-12-04 02:37:00,Ed Maverick,Acurrucar,70.2,00:01:10.200000,2019-12-04 02:35:49.800
4,2019-12-04 06:05:00,J. Cole,G.O.M.D.,2.4,00:00:02.400000,2019-12-04 06:04:57.600


In [31]:
df_2020 = df.loc[(df['startTime'] >= "2020-01-01") & (df['endTime'] <= "2020-12-31")]t
df_2020.head()

,endTime,artistName,trackName,secPlayed,secPlayed_dt,startTime
1144,2020-01-01 06:13:00,Arizona Zervas,ROXANNE,163.6,00:02:43.600000,2020-01-01 06:10:16.400
1145,2020-01-01 06:13:00,Olivia Rodrigo,"All I Want - From ""High School Musical: The Mu...",6.4,00:00:06.400000,2020-01-01 06:12:53.600
1146,2020-01-01 06:14:00,H E R B,Save Me,25.7,00:00:25.700000,2020-01-01 06:13:34.300
1147,2020-01-02 14:19:00,Kanye West,No More Parties In LA,106.6,00:01:46.600000,2020-01-02 14:17:13.400
1148,2020-01-02 16:34:00,Meek Mill,Dreams and Nightmares,1.8,00:00:01.800000,2020-01-02 16:33:58.200


In [ ]:
def get_spotify_uri(song):
    """Returns the corresponding spotify URI from a given song title"""       
    try:
        search_results = sp.search(q=song, type='track', limit=1)
        track_id = search_results['tracks']['items'][0]['id']
    except (AttributeError, IndexError) as err:
        print('No results for {}'.format(song))
        pass

In [33]:
pd.read_csv("track_uris.csv")

ParserError: Error tokenizing data. C error: Expected 2 fields in line 95, saw 3


In [36]:
df.index[:100]

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
            68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
            85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
           dtype='int64')